In [1]:
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
#imgfolder = '/home/dane/data/imdb_crop/'
#meta = '/home/dane/data/imdb_crop/meta.csv'
imgfolder = '/data/nlp/corpora/imdb_crop/'
meta = '/data/nlp/corpora/imdb_crop/meta.csv'
height = 224
width = 224
channels = 3

annos = pd.DataFrame.from_csv(meta, header=None, index_col=None)
annos.columns = ['age', 'gender', 'file']
annos = annos.dropna()

In [3]:
ids = list(set(annos.file))
ids = np.random.permutation(ids)
boundary = round(0.9 * len(ids))
train_ids = ids[0:boundary]
test_ids = ids[boundary:]

In [4]:
train_imgs = []
train_labels = []
test_imgs = []
test_labels = []

for tid in train_ids[0:int(.001 * len(train_ids))]:
    label = annos.gender[annos.file == tid].asobject[0]
    if not pd.isnull(label):
        img_path = imgfolder + tid
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        train_imgs.append(x)
        train_labels.append(label)
    else:
        print(annos.loc[annos[2] == tid])
    
for tid in test_ids[0:int(.001 * len(test_ids))]:
    label = annos.gender[annos.file == tid].asobject[0]
    if not pd.isnull(label):
        img_path = imgfolder + tid
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        test_imgs.append(x)
        test_labels.append(label)
    else:
        print(annos.loc[annos[2] == tid])




In [5]:
train_labels = np_utils.to_categorical(train_labels, 2)
test_labels = np_utils.to_categorical(test_labels, 2)

In [6]:
img_rows, img_cols, img_channels = height, width, channels

X_train = np.array(train_imgs)
X_test = np.array(test_imgs)
if K.image_dim_ordering() == 'tf':
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, channels)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, channels)
    input_shape = input_shape = (img_rows, img_cols, channels)
else:
    input_shape = (channels, img_rows, img_cols)
    
y_train = np.array(train_labels)
y_test = np.array(test_labels)


In [8]:
base_model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, kernel_initializer='normal', activation='relu')(x)
x = Dense(2, kernel_initializer='normal', activation='softmax')(x)

model = Model(inputs = base_model.inputs, outputs = x)

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])


In [9]:
batch_size = 5
nb_epoch = 1

model.fit(X_train, y_train, batch_size=batch_size, epochs=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 407 samples, validate on 45 samples
Epoch 1/1
 90/407 [=====>........................] - ETA: 14:54 - loss: 6.1730 - acc: 0.6000

KeyboardInterrupt: 